In [ ]:
!pip install transformers==4.55.2 peft==0.17.0 accelerate==1.10.0 trl==0.21.0 bitsandbytes==0.47.0 datasets==4.0.0 huggingface-hub==0.34.4 safetensors==0.6.2 pandas==2.2.2 matplotlib==3.10.0 numpy==2.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.8/485.8 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.8 MB/s eta 0:00:00
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.7.0
    Uninstalling safetensors-0.7.0:
      Successfully uninstalled safetensors-0.7.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstal

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    BitsAndBytesConfig,
    Trainer)
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
# import evaluate
from trl import SFTConfig, SFTTrainer


import torch
import numpy as np
import os

In [ ]:
bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.float32
)
# repo_id = "meta-llama/Llama-3.2-3B-Instruct"
repo_id = "microsoft/Phi-3-mini-4k-instruct"
model = AutoModelForCausalLM.from_pretrained(
  repo_id, device_map="auto", quantization_config=bnb_config
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
print(model.get_memory_footprint()/1e6)
model = prepare_model_for_kbit_training(model)
# model.unload()
config = LoraConfig(
    r=12,
    lora_alpha=16,
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",

    target_modules=['o_proj', 'qkv_proj', 'gate_up_proj', 'down_proj'],
)
model = get_peft_model(model, config)

2600.743104


In [ ]:
train_p, tot_p = model.get_nb_trainable_parameters()
print(f'Trainable parameters:      {train_p/1e6:.2f}M')
print(f'Total parameters:          {tot_p/1e6:.2f}M')
print(f'% of trainable parameters: {100*train_p/tot_p:.2f}%')

Trainable parameters:      18.87M
Total parameters:          3839.95M
% of trainable parameters: 0.49%


In [ ]:
from google.colab import files
files.upload()

Saving data.json to data.json


{'data.json': b'[\r\n  {\r\n    "label": "Loan Information",\r\n    "question": "What is a personal loan?",\r\n    "answer": "A personal loan is an unsecured loan that can be used for various personal expenses such as medical costs, travel, or home improvements. Approval and terms depend on the bank\xe2\x80\x99s eligibility criteria.",\r\n    "reasoning": "This response provides a correct definition and avoids guarantees, reflecting standard banking explanations."\r\n  },\r\n  {\r\n    "label": "Loan Eligibility",\r\n    "question": "Who is eligible to apply for a personal loan?",\r\n    "answer": "Eligibility for a personal loan is generally based on factors such as income stability, credit history, employment status, and repayment capacity.",\r\n    "reasoning": "The answer mentions realistic eligibility factors without promising approval."\r\n  },\r\n  {\r\n    "label": "Interest Rate Inquiry",\r\n    "question": "What determines the interest rate on a loan?",\r\n    "answer": "Loan

In [ ]:
import pandas as pd
import json

In [ ]:
with open("data.json","r") as d:
  raw = json.load(d)

In [ ]:

def to_model_format(examples):
  formatted_data = []
  for example in examples:
    formatted_data.append({
      "messages":[
          {"role":"user","content":example['question']},
          {"role":"assistant","content":f"{example['answer']} [Reasoning : {example['reasoning']}]" }
                  ]
    })
  return formatted_data
          # {"role":"system","content":"Reasoning is for internal training. Dont tell to user untill user explicitly tells you"},

formatData = to_model_format(raw)

with open("data.jsonl","w") as f:
  for ex in formatData:
    f.write(json.dumps(ex,ensure_ascii=False)+"\n")
with open("data.jsonl","r") as r:
  print(r.read())

{"messages": [{"role": "user", "content": "What is a personal loan?"}, {"role": "assistant", "content": "A personal loan is an unsecured loan that can be used for various personal expenses such as medical costs, travel, or home improvements. Approval and terms depend on the bank’s eligibility criteria. [Reasoning : This response provides a correct definition and avoids guarantees, reflecting standard banking explanations.]"}]}
{"messages": [{"role": "user", "content": "Who is eligible to apply for a personal loan?"}, {"role": "assistant", "content": "Eligibility for a personal loan is generally based on factors such as income stability, credit history, employment status, and repayment capacity. [Reasoning : The answer mentions realistic eligibility factors without promising approval.]"}]}
{"messages": [{"role": "user", "content": "What determines the interest rate on a loan?"}, {"role": "assistant", "content": "Loan interest rates are determined by factors such as the applicant’s credi

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(repo_id)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id
tokenizer.chat_template

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

"{% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'user' %}{{'<|user|>\n' + message['content'] + '<|end|>\n'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>\n' + message['content'] + '<|end|>\n'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>\n' }}{% else %}{{ eos_token }}{% endif %}"

In [ ]:
sft_config = SFTConfig(
    ## GROUP 1: Memory usage
    # These arguments will squeeze the most out of your GPU's RAM
    # Checkpointing
    gradient_checkpointing=True,    # this saves a LOT of memory
    # Set this to avoid exceptions in newer versions of PyTorch
    gradient_checkpointing_kwargs={'use_reentrant': False},
    # Gradient Accumulation / Batch size
    # Actual batch (for updating) is same (1x) as micro-batch size
    gradient_accumulation_steps=1,
    # The initial (micro) batch size to start off with
    per_device_train_batch_size=16,
    # If batch size would cause OOM, halves its size until it works
    auto_find_batch_size=True,

    ## GROUP 2: Dataset-related
    max_length=64, # renamed in v0.20
    # Dataset
    # packing a dataset means no padding is needed
    packing=True,
    packing_strategy='wrapped', # added to approximate original packing behavior

    ## GROUP 3: These are typical training parameters
    num_train_epochs=10,    learning_rate=3e-4,
    # Optimizer
    # 8-bit Adam optimizer - doesn't help much if you're using LoRA!
    optim='paged_adamw_8bit',

    ## GROUP 4: Logging parameters
    logging_steps=10,
    logging_dir='./logs',
    output_dir='./phi3-mini-yoda-adapter',
    report_to='none',

    # ensures bf16 (the new default) is only used when it is actually available
    bf16 = torch.cuda.is_bf16_supported(including_emulation=False))

In [ ]:
dataset = load_dataset("json",data_files="data.jsonl")["train"]
split = dataset.train_test_split(test_size=0.11)
trainData = split['train']
testData = split['test']
print(trainData)

Dataset({
    features: ['messages'],
    num_rows: 765
})


In [ ]:
trainer = SFTTrainer(
    model=model.base_model.model, # the underlying Phi-3 model
    peft_config=config,  # added to fix issue in TRL>=0.20
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=trainData,
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/765 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/765 [00:00<?, ? examples/s]

In [ ]:
dl = trainer.get_train_dataloader()
batch = next(iter(dl))

In [ ]:
batch['input_ids'][0], batch['labels'][0]

(tensor([29889, 32007, 32001,  3869, 29892,   565,   596, 23562,   508,  4386,
           372, 29889,  1205,  5622,  2086,  1784,   658,   550,   508,   367,
          5161,  3459, 29889,   518,  1123,  1658,   292,   584,  5947,   414,
           274,  1300,  2738, 27323,  5586, 32007, 32000, 32010, 12142,   306,
          4459, 24332,  1286,   541, 15982,  1048,   278,  5434, 29892,   338,
           393,  4226, 29973, 32007, 32001,  3869, 29892,   372, 30010, 29879,
          4226, 29889,  1858,  9450], device='cuda:0'),
 tensor([29889, 32007, 32001,  3869, 29892,   565,   596, 23562,   508,  4386,
           372, 29889,  1205,  5622,  2086,  1784,   658,   550,   508,   367,
          5161,  3459, 29889,   518,  1123,  1658,   292,   584,  5947,   414,
           274,  1300,  2738, 27323,  5586, 32007, 32000, 32010, 12142,   306,
          4459, 24332,  1286,   541, 15982,  1048,   278,  5434, 29892,   338,
           393,  4226, 29973, 32007, 32001,  3869, 29892,   372, 30010, 298

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,2.686700
20,1.811700
30,1.657900
40,1.539400
50,1.367200
60,1.307300
70,1.291300
80,1.256400
90,0.983700
100,0.909900


TrainOutput(global_step=400, training_loss=0.6137626448273659, metrics={'train_runtime': 2823.0495, 'train_samples_per_second': 2.256, 'train_steps_per_second': 0.142, 'total_flos': 9151894117416960.0, 'train_loss': 0.6137626448273659})

In [ ]:
from google.colab import drive
# drive.mount("/content/drive")?
trainer.save_model("/content/phi3-lora-chatbot")

In [ ]:
!zip -r phi3-lora-chatbot.zip phi3-lora-chatbot

  adding: phi3-lora-chatbot/ (stored 0%)
  adding: phi3-lora-chatbot/adapter_model.safetensors (deflated 7%)
  adding: phi3-lora-chatbot/special_tokens_map.json (deflated 72%)
  adding: phi3-lora-chatbot/added_tokens.json (deflated 62%)
  adding: phi3-lora-chatbot/adapter_config.json (deflated 56%)
  adding: phi3-lora-chatbot/tokenizer_config.json (deflated 86%)
  adding: phi3-lora-chatbot/training_args.bin (deflated 53%)
  adding: phi3-lora-chatbot/chat_template.jinja (deflated 60%)
  adding: phi3-lora-chatbot/README.md (deflated 65%)
  adding: phi3-lora-chatbot/tokenizer.model (deflated 55%)
  adding: phi3-lora-chatbot/tokenizer.json (deflated 85%)


In [ ]:
from google.colab import files
files.download("phi3-lora-chatbot.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
eval_results = trainer.evaluate(eval_dataset=testData)
print(eval_results)

ValueError: No columns in the dataset match the model's forward method signature: (input_ids, labels, seq_lengths, completion_mask, assistant_masks). The following columns have been ignored: [messages]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.